In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('../processed/train.csv', index_col='PassengerId')
test_df = pd.read_csv('../processed/test.csv', index_col='PassengerId')
test_df = test_df.drop(['Survived'], axis=1)

In [3]:
train_df.head()

,Survived,Age,Fare,FamilySize,IsMale,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,...,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Sir,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,22.0,7.2500,2,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
2,1,38.0,71.2833,2,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
3,1,26.0,7.9250,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
4,1,35.0,53.1000,2,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,1,0
5,0,35.0,8.0500,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


### using mean shift cluster

In [23]:
from sklearn.cluster import MeanShift

analyzer = MeanShift(bandwidth=28)

analyzer.fit(train_df)

MeanShift(bandwidth=28, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)

In [24]:
#getting the bandwidth estimation
from sklearn.cluster import estimate_bandwidth
estimate_bandwidth(train_df)

27.87900269534258

In [25]:
labels = analyzer.labels_

np.unique(labels) 

array([0, 1, 2, 3, 4, 5])

In [13]:
# a bandwidth of 50 produces 3 clusters i.e threee groups of passengers with uniques characteristics


In [26]:
 # add cluster groups 

train_df['cluster_group'] = np.nan
data_length = len(train_df)

for i in range(data_length):
    train_df.iloc[i, train_df.columns.get_loc('cluster_group')] = labels[i]

In [27]:
train_df.describe()

,Survived,Age,Fare,FamilySize,IsMale,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,...,Title_Mr,Title_Mrs,Title_Officer,Title_Sir,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child,cluster_group
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,29.311639,32.204208,1.904602,0.647587,0.016835,0.052750,0.066218,0.037037,0.035915,...,0.580247,0.141414,0.005612,0.010101,0.190797,0.086420,0.722783,0.868687,0.131313,0.316498
std,0.486592,13.243668,49.693429,1.613459,0.477990,0.128725,0.223659,0.248802,0.188959,0.186182,...,0.493796,0.348644,0.074743,0.100051,0.393150,0.281141,0.447876,0.337932,0.337932,0.703720
min,0.000000,0.420000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,22.000000,7.910400,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,0.000000,29.000000,14.454200,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
75%,1.000000,35.500000,31.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,80.000000,512.329200,11.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000


In [28]:
train_cluster_data = train_df.groupby(['cluster_group']).mean()

train_cluster_data

,Survived,Age,Fare,FamilySize,IsMale,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,...,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Sir,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
cluster_group,,,,,,,,,,,,,,,,,,,,,
0.0,0.321377,28.201937,14.265111,1.674319,0.690100,0.008608,0.011478,0.021521,0.015782,0.021521,...,0.193687,0.626973,0.114778,0.004304,0.011478,0.136298,0.107604,0.756098,0.868006,0.131994
1.0,0.541353,33.827068,60.971932,2.842105,0.556391,0.067669,0.150376,0.127820,0.142857,0.112782,...,0.210526,0.466165,0.233083,0.015038,0.007519,0.330827,0.000000,0.669173,0.864662,0.135338
2.0,0.780488,32.729756,123.310463,2.341463,0.341463,0.000000,0.243902,0.463415,0.073171,0.048780,...,0.365854,0.243902,0.292683,0.000000,0.000000,0.463415,0.048780,0.487805,0.853659,0.146341
3.0,0.666667,26.600000,235.910833,2.933333,0.333333,0.000000,0.400000,0.466667,0.000000,0.000000,...,0.533333,0.333333,0.133333,0.000000,0.000000,0.533333,0.000000,0.466667,0.933333,0.066667
4.0,1.000000,35.333333,512.329200,1.333333,0.666667,0.000000,0.666667,0.000000,0.000000,0.000000,...,0.333333,0.666667,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
5.0,0.500000,57.000000,255.260400,4.000000,0.500000,0.000000,0.500000,0.500000,0.000000,0.000000,...,0.000000,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000,0.500000,1.000000,0.000000


In [29]:
train_cluster_data['counts'] = pd.Series(train_df.groupby(['cluster_group']).size())

train_cluster_data

,Survived,Age,Fare,FamilySize,IsMale,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,...,Title_Mr,Title_Mrs,Title_Officer,Title_Sir,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child,counts
cluster_group,,,,,,,,,,,,,,,,,,,,,
0.0,0.321377,28.201937,14.265111,1.674319,0.690100,0.008608,0.011478,0.021521,0.015782,0.021521,...,0.626973,0.114778,0.004304,0.011478,0.136298,0.107604,0.756098,0.868006,0.131994,697
1.0,0.541353,33.827068,60.971932,2.842105,0.556391,0.067669,0.150376,0.127820,0.142857,0.112782,...,0.466165,0.233083,0.015038,0.007519,0.330827,0.000000,0.669173,0.864662,0.135338,133
2.0,0.780488,32.729756,123.310463,2.341463,0.341463,0.000000,0.243902,0.463415,0.073171,0.048780,...,0.243902,0.292683,0.000000,0.000000,0.463415,0.048780,0.487805,0.853659,0.146341,41
3.0,0.666667,26.600000,235.910833,2.933333,0.333333,0.000000,0.400000,0.466667,0.000000,0.000000,...,0.333333,0.133333,0.000000,0.000000,0.533333,0.000000,0.466667,0.933333,0.066667,15
4.0,1.000000,35.333333,512.329200,1.333333,0.666667,0.000000,0.666667,0.000000,0.000000,0.000000,...,0.666667,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,3
5.0,0.500000,57.000000,255.260400,4.000000,0.500000,0.000000,0.500000,0.500000,0.000000,0.000000,...,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000,0.500000,1.000000,0.000000,2


In [30]:
train_df[train_df.cluster_group == 1]

,Survived,Age,Fare,FamilySize,IsMale,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,...,Title_Mr,Title_Mrs,Title_Officer,Title_Sir,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child,cluster_group
PassengerId,,,,,,,,,,,,,,,,,,,,,
2,1,38.0,71.2833,2,0,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,1.0
4,1,35.0,53.1000,2,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,1.0
7,0,54.0,51.8625,1,1,0,0,0,0,1,...,1,0,0,0,0,0,1,1,0,1.0
35,0,28.0,82.1708,2,1,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,1.0
36,0,42.0,52.0000,2,1,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,1.0
44,1,3.0,41.5792,4,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1.0
51,0,7.0,39.6875,6,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1.0
53,1,49.0,76.7292,2,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,1.0
55,0,65.0,61.9792,2,1,0,1,0,0,0,...,1,0,0,0,1,0,0,1,0,1.0
